###### CELLO Mk 3.0, EST. 2023-3-3


## COMPONENT #2:
## Restricted Graph Conversion to Python netlist object

SAVES the Netlist JSON file produced by YOSYS as an OBJECT such that we can map and evaluates parts to it

In [1]:
import json

In [2]:
in_path = '../../IO/inputs'
out_path = '../../IO/temp_folder'

In [3]:
# ENTRY POINT
netlist_name = 'xor'

In [4]:
netlist_path = out_path + '/' + netlist_name + '.json'

In [5]:
netlist_file = open(netlist_path, 'r')
netlist = json.load(netlist_file)

# important: netlist has in the json format generated by YOSYS
# TODO: Add conversion from other netlist types (such as STL)

In [6]:
# test print the entire netlist
print(json.dumps(netlist, indent=4))

{
    "creator": "Yosys 0.25 (git sha1 e02b7f64bc7, clang 14.0.0 -fPIC -Os)",
    "modules": {
        "xor_gate": {
            "attributes": {
                "top": "00000000000000000000000000000001",
                "src": "../../IO/inputs/xor.v:4.1-18.10"
            },
            "ports": {
                "a": {
                    "direction": "input",
                    "bits": [
                        2
                    ]
                },
                "b": {
                    "direction": "input",
                    "bits": [
                        3
                    ]
                },
                "out": {
                    "direction": "output",
                    "bits": [
                        4
                    ]
                }
            },
            "cells": {
                "$abc$80$auto$blifparse.cc:386:parse_blif$81": {
                    "hide_name": 1,
                    "type": "$_NOT_",
                    "parameters": {}

In [7]:
# print(len(netlist))
# # 2 is right
circuit_name = list(netlist["modules"].keys())[0]
print("NAME: " + str(circuit_name))

NAME: xor_gate


In [8]:
components = []
# dev res/s
for m in netlist["modules"][circuit_name]:
    components.append(m)
    print(m)
print()

if 'parameter_default_values' in components:
    print("we've got an extra component here! \n")
    print(json.dumps(netlist['modules'][circuit_name]['parameter_default_values'], indent=4))
else:
    print('we\'re good')

attributes
ports
cells
netnames

we're good


In [9]:
# temporarily define the componenets of the netlist as vars
ports = netlist["modules"][circuit_name]['ports']
cls = netlist["modules"][circuit_name]['cells']
edges = netlist["modules"][circuit_name]['netnames']

# scan for special parameters and attributes
for gatename, gatedata in cls.items():
    if gatedata['parameters']:
        print(f"GATE '{gatename}' has PARAMETER '{gatedata['parameters']}'")
    elif gatedata['attributes']:
        print(f"GATE '{gatename}' has ATTRIBUTE '{gatedata['attributes']}'")

for netname, netdata in edges.items():
    if 'src' in netdata['attributes']:
        print(f"NET '{netname}' has SOURCE '{netdata['attributes']['src']}'")

NET 'a' has SOURCE '../../IO/inputs/xor.v:11.10-11.11'
NET 'b' has SOURCE '../../IO/inputs/xor.v:12.10-12.11'
NET 'out' has SOURCE '../../IO/inputs/xor.v:14.11-14.14'


In [10]:
'''
_sort_gates(self, cells)

'''

for c in cls.keys():
    partition = list(c.split('$'))
    gate_id = partition[-1]
    gate_type = cls[c]['type']
#     print(gate_id + ' ' + gate_type + ' ' + partition[1])
    gate = {gate_id : 
            {
             'type' : gate_type,
             'inputs' : {},
             'output' : {},
            }
           }
#     print(gate)
#     print(json.dumps(gate, indent=4))
#     print()
    dirns = cls[c]['port_directions']
    ctns = cls[c]['connections']
    inputs = []
    outputs = []
    for d in dirns.keys():
        inout = dirns[d]
        if inout == 'input':
            inputs.append(d)
        else:
            outputs.append(d)
    gate_inputs = []
    gate_outputs = []
    for c in ctns.keys():
        c_nodes = ctns[c]
        if c in inputs:
            gate_inputs.append((c, c_nodes))
        else:
            gate_outputs.append((c, c_nodes))
#     gate[gate_id]['inputs'] = gate_inputs
#     gate[gate_id]['output'] = gate_outputs
#     print(gate_inputs)
#     print(gate_outputs)
    for tup in gate_inputs:
        (node_name, edge_nos) = tup
        try:
            gate[gate_id]['inputs'][node_name] += edge_nos
        except Exception as e:
            gate[gate_id]['inputs'][node_name] = edge_nos
            # print('creating node: ' + str(e))
    for tup in gate_outputs:
        (node_name, edge_nos) = tup
        try:
            gate[gate_id]['output'][node_name] += edge_nos
        except Exception as e:
            gate[gate_id]['output'][node_name] = edge_nos
            # print('creating node: ' + str(e))
    # print(json.dumps(gate, indent=4))
#         print(d + ' ' + str(dirns[d]))
#     for n in ctns.keys():
print("pass")

pass


In [11]:


in_nodes = []
out_nodes = []
for p in ports.keys():
    node_name = p
    direction = ports[p]['direction']
    bits = ports[p]['bits']
    node = (node_name, bits)
    if direction == 'input':
        in_nodes.append(node)
    elif direction == 'output':
        out_nodes.append(node)
    else:
        raise ValueError('Invalid [in/out]put node') 
print("inputs: \n" + str(in_nodes) + '\n')
print("outputs: \n" + str(out_nodes))

inputs: 
[('a', [2]), ('b', [3])]

outputs: 
[('out', [4])]


In [12]:
class Netlist:
    def __init__(self, netlistJSON):
        net_main = netlistJSON['modules']
        self.name = list(net_main.keys())[0]
        ports = net_main[self.name]['ports']
        cells = net_main[self.name]['cells']
        edges = net_main[self.name]['netnames']
        i, o = self.__sort_nodes(ports)
        self.inputs = i
        self.outputs = o
        self.gates = self.__sort_gates(cells)
        self.edges = self.__check_edges(edges)
        
    def __sort_nodes(self, ports):
        in_nodes = []
        out_nodes = []
        for p in ports.keys():
            node_name = p
            direction = ports[p]['direction']
            bits = ports[p]['bits']
            node = (node_name, bits)
            if direction == 'input':
                in_nodes.append(node)
            elif direction == 'output':
                out_nodes.append(node)
            else:
                raise ValueError('Invalid [in/out]put node') 
        return (in_nodes, out_nodes)
    
    def __sort_gates(self, cells):
        gates = {}
        for c in cells.keys():
            partition = list(c.split('$'))
            gate_id = partition[-1]
            gate_type = cells[c]['type']
            gate = {gate_id : 
                    {
                     'type' : gate_type,
                     'inputs' : {},
                     'output' : {},
                    }
                   }
            dirns = cells[c]['port_directions']
            ctns = cells[c]['connections']
            inputs = []
            outputs = []
            for d in dirns.keys():
                inout = dirns[d]
                if inout == 'input':
                    inputs.append(d)
                else:
                    outputs.append(d)
            gate_inputs = []
            gate_outputs = []
            for c in ctns.keys():
                c_nodes = ctns[c]
                if c in inputs:
                    gate_inputs.append((c, c_nodes))
                else:
                    gate_outputs.append((c, c_nodes))
            for tup in gate_inputs:
                (node_name, edge_nos) = tup
                try:
                    gate[gate_id]['inputs'][node_name] += edge_nos
                except Exception as e:
                    gate[gate_id]['inputs'][node_name] = edge_nos
            for tup in gate_outputs:
                (node_name, edge_nos) = tup
                try:
                    gate[gate_id]['output'][node_name] += edge_nos
                except Exception as e:
                    gate[gate_id]['output'][node_name] = edge_nos
            gates.update(gate)
        return gates
    
    def __str__(self):
        return (f"{self.name}, with: \n"
                f"{len(self.inputs)} inputs,\n"
                f"{len(self.outputs)} ouputs.")
    
    def __check_edges(self, edges):
        for n in edges:
            pass
        return 9

In [13]:
testNet = Netlist(netlist)

In [14]:
print(testNet.inputs)

[('a', [2]), ('b', [3])]


In [15]:
print(testNet.outputs)

[('out', [4])]


In [16]:
print(json.dumps(testNet.gates, indent=4))

{
    "81": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                2
            ]
        },
        "output": {
            "Y": [
                5
            ]
        }
    },
    "82": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                3
            ]
        },
        "output": {
            "Y": [
                6
            ]
        }
    },
    "83": {
        "type": "$_NOR_",
        "inputs": {
            "A": [
                2
            ],
            "B": [
                3
            ]
        },
        "output": {
            "Y": [
                7
            ]
        }
    },
    "84": {
        "type": "$_NOR_",
        "inputs": {
            "A": [
                5
            ],
            "B": [
                6
            ]
        },
        "output": {
            "Y": [
                8
            ]
        }
    },
    "85": {
        "type": "$_NOR_",
        "inputs": {
    

In [17]:
print(testNet)

xor_gate, with: 
2 inputs,
1 ouputs.
